In [ ]:
!git clone https://github.com/raquelaoki/ParKCa.git
!git clone https://github.com/raquelaoki/CompBioAndSimulated_Datasets.git
!git clone https://github.com/JakeColtman/bartpy.git

Cloning into 'ParKCa'...
remote: Enumerating objects: 488, done.
remote: Counting objects: 100% (488/488), done.
remote: Compressing objects: 100% (341/341), done.
remote: Total 1084 (delta 336), reused 272 (delta 145), pack-reused 596
Receiving objects: 100% (1084/1084), 3.21 MiB | 10.30 MiB/s, done.
Resolving deltas: 100% (716/716), done.
Cloning into 'CompBioAndSimulated_Datasets'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 108 (delta 64), reused 66 (delta 29), pack-reused 0
Receiving objects: 100% (108/108), 127.34 KiB | 6.70 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Cloning into 'bartpy'...
remote: Enumerating objects: 1971, done.
remote: Total 1971 (delta 0), reused 0 (delta 0), pack-reused 1971
Receiving objects: 100% (1971/1971), 11.97 MiB | 2.15 MiB/s, done.
Resolving deltas: 100% (1329/1329), done.


In [ ]:
import random
import pandas as pd
import numpy as np
import sys
import torch
sys.path.insert(0, 'src/')
sys.path.insert(0,'bartpy/')
sys.path.insert(0,'ParKCa/src/')
from ParKCa.src.train import *
from CompBioAndSimulated_Datasets.simulated_data_multicause import *
from sklearn.model_selection import train_test_split


Instructions for updating:
non-resource variables are not supported in the long term


Loading data

In [ ]:
import random
import pandas as pd
import numpy as np
import sys
import torch
sys.path.insert(0, 'src/')
sys.path.insert(0,'bartpy/')
sys.path.insert(0,'ParKCa/src/')
from ParKCa.src.train import *
from CompBioAndSimulated_Datasets.simulated_data_multicause import *
from sklearn.model_selection import train_test_split

def trykey(params,key,default):
    try:
        return params[key]
    except KeyError:
        params[key] = default
        return params[key]



m3e2 = True
if m3e2: 

  import time
  import yaml
  import model_m3e2 as m3e2
  #import torch.nn as nn
  #import torch.nn.functional as F
  #from torch.utils.data import Dataset, DataLoader, TensorDataset
  #from torch import optim, Tensor


  def main(config_path):
      """Start: Parameters Loading"""
      with open(config_path) as f:
          config = yaml.safe_load(f)
      params = config['parameters']

      try:
          SEED = params["SEED"]
      except KeyError:
          SEED = 10

      # Fix numpy seed for reproducibility
      np.random.seed(SEED)
      # Fix random seed for reproducibility
      random.seed(SEED)
      # Fix Torch graph-level seed for reproducibility
      torch.manual_seed(SEED)
      print('seed',SEED)

      if 'gwas' in params['data']:
          sdata_gwas = gwas_simulated_data(prop_tc=0.05, pca_path='/content/CompBioAndSimulated_Datasets/data/tgp_pca2.txt')
          X, y, y01, treatement_columns, treatment_effects, group = sdata_gwas.generate_samples()
          X_train, X_test, y_train, y_test = train_test_split(X, y01, test_size=0.33, random_state=SEED)
          print('... Target - proportion of 1s', np.sum(y01)/len(y01) )
          # Split X1, X2 on GWAS
          X1_cols = []
          X2_cols = range(X.shape[1]-len(treatement_columns))
          # TODO: add other baselines here to run everything on the same train/testing sets

          data_nnl = m3e2.data_nn(X_train.values, X_test.values, y_train, y_test, treatement_columns,
                                  treatment_effects[treatement_columns], X1_cols, X2_cols)
          loader_train, loader_val, loader_test, num_features = data_nnl.loader(params['suffle'],params['batch_size'],SEED)
          params['pos_weights'] = data_nnl.treat_weights
          params['pos_weight_y'] = trykey(params,'pos_weight_y',1)
          params['hidden1'] = trykey(params,'hidden1',64)
          params['hidden2'] = trykey(params,'hidden2',8)
          cate_m3e2 = m3e2.fit_nn(loader_train, loader_val, loader_test, params, treatement_columns, num_features,
                                X1_cols, X2_cols)
          print('... CATE')
          cate = pd.DataFrame({'CATE_M3E2':cate_m3e2,'True_Effect':treatment_effects[treatement_columns]})
          print(cate)
          dif = cate_m3e2-treatment_effects[treatement_columns]
          print('MAE',np.abs(dif).mean())
      if 'copula' in params['data']:
          sdata_copula = copula_simulated_data()
          X, y, y01, treatement_columns, treatment_effects = sdata_copula.generate_samples()
          X_train, X_test, y_train, y_test = train_test_split(X, y01, test_size=0.33, random_state=SEED)
          print('... Target - proportion of 1s', np.sum(y01) / len(y01))
          # Split X1, X2 on GWAS
          X1_cols = []
          X2_cols = range(X.shape[1] - len(treatement_columns))
          # TODO: add other baselines here to run everything on the same train/testing sets
          data_nnl = m3e2.data_nn(X_train, X_test, y_train, y_test, treatement_columns,
                                  treatment_effects, X1_cols, X2_cols)
          loader_train, loader_val, loader_test, num_features = data_nnl.loader(params['suffle'], params['batch_size'],
                                                                                SEED)
          params['pos_weights'] = data_nnl.treat_weights
          params['pos_weight_y'] = trykey(params,'pos_weight_y',1)
          params['hidden1'] = trykey(params,'hidden1',6)
          params['hidden2'] = trykey(params,'hidden2',3)
          cate_m3e2 = m3e2.fit_nn(loader_train, loader_val, loader_test, params, treatement_columns, num_features,
                                  X1_cols, X2_cols)
          print('... CATE')
          cate = pd.DataFrame({'CATE_M3E2': cate_m3e2, 'True_Effect': treatment_effects})
          print(cate)
          dif = cate_m3e2 - treatment_effects
          print('MAE', np.abs(dif).mean())
      if 'gwas' not in params['data'] and 'copula' not in params['data']:
          print('ERRROR! \nDataset not recognized. \nChange the parameter data in your config.yaml file to gwas or copula.')  

  if __name__ == "__main__":
      start_time = time.time()
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      print("Cuda Availble:", torch.cuda.is_available(), " device: ", device)
      main(config_path='/content/config2.yaml')
      end_time = time.time() - start_time
      end_time_m = end_time / 60
      end_time_h = end_time_m / 60
      print("Time ------ {} min / {} hours ------".format(end_time_m, end_time_h))

Instructions for updating:
non-resource variables are not supported in the long term
Cuda Availble: True  device:  cuda
seed 10
Copula simulated data initialized!
... Treatments: (10000, 4)
... Confounders: (10000, 10)
... Target (y): 0.6766
Data Simulation Done!
[[-1.   -1.    1.   -0.06]]
[[ 1.02752294  0.25688073 -0.20550459  0.10275229]]
... Target - proportion of 1s 0.6766
Weights [ -33.79254788 -635.89574239    8.07677019  -50.02857128]
M3E2: Train Shape  (6700, 10) (6700, 4)
... Model initialization done!
... Training
......  0  
Train: loss  27.78 1.12 metric  [4.92599201 1.20046854 0.81006777 1.1302768  0.61151283] 
Val: loss  17.24 metric  [5.16339493 1.26362026 0.94801182 1.09876001 0.5961962 ]
......  30  
Train: loss  18.17 1.01 metric  [2.6418469  1.08863413 0.74769765 1.0547576  0.67761764] 
Val: loss  11.76 metric  [2.62610793 1.15354431 0.75240564 0.99653137 0.66956957]
......  60  
Train: loss  16.64 0.96 metric  [2.24643588 1.1452136  0.72321409 1.07581091 0.64684186

TODO: 
1. in copula, treat is continuous - fine for DA, need change our loss/propensity; 
2. A treatment is dominating the training set
3. Add RMSE with DA, BART, CEVAE results for GWAS, copula. 
  a.Create a table, single x multi, with empty spaces for future models 
  b. What I'm evaluating? Adding more confoudners robusteness, adding more treatments (T) (add running time tracker), adding more samples (n). 
4. Plots 

In [ ]:
def baselines(BaselinesList, X, y, ParamsList, seed=63, TreatCols=None, id='', timeit=False):
    """
    input:
        X, colnamesX: potential causes and their names
        Z, colnamesZ: confounders and their names
        y: 01 outcome
        causes: name of the potential causes
    """
    if TreatCols is None:
        TreatCols = list(range(X.shape[1]))

    # check if binary treatments
    X01 = X.copy()
    for col in TreatCols:
        a = X01.iloc[:,col]
        if not ((a == 0) | (a == 1)).all():
            mean_v = np.mean(X01.iloc[:,col])
            X01.iloc[:,col] = [1 if i > mean_v else 0 for i in X01.iloc[:,col]]
        else:
            pass

    X_train, X_test, y_train, y_test, X_train01, X_test01 = train_test_split(X, y, X01,
                                                                             test_size=0.33, random_state=seed)
    coef_table = pd.DataFrame(columns=['causes'])
    coef_table['causes'] = ['T' + str(i) for i in range(len(TreatCols))]
    times = {}

    if 'DA' in BaselinesList:
        print('\n\nBaseline: DA')
        start_time = time.time()
        from deconfounder import deconfounder_algorithm as DA
        ParamsList['DA']['k'] = trykey(ParamsList['DA'],'k',15) # if exploring multiple latent sizes
        for k in ParamsList['DA']['k']:
            if len(ParamsList['DA']['k']) > 1:
                coln = 'DA_' + str(id) + str(k)
            else:
                coln = 'DA'
            model_da = DA(X_train, X_test, y_train, y_test, k)
            ParamsList['DA']['class_weight'] = trykey(ParamsList['DA'], 'class_weight', {0:1,1:1})
            coef, coef_continuos, roc = model_da.fit(class_weight=ParamsList['DA']['class_weight'])
            coef_table[coln] = coef_continuos[0:len(TreatCols)]
        times['DA'] = time.time() - start_time
        print('Done!')

    if 'BART' in BaselinesList:
        print('\n\nLearner: BART')
        start_time = time.time()
        from bart import BART as BART
        model_bart = BART(X_train01, X_test01, y_train, y_test)
        ParamsList['BART']['n_trees'] = trykey(ParamsList['BART'], 'n_trees', 50)
        ParamsList['BART']['n_burn'] = trykey(ParamsList['BART'], 'n_burn', 100)
        model_bart.fit(n_trees=ParamsList['BART']['n_trees'], n_burn=ParamsList['BART']['n_burn'])
        print('...... predictions')
        coef_table['BART'] = model_bart.cate(TreatCols)
        times['BART'] = time.time() - start_time
        print('Done!')

    if 'CEVAE' in BaselinesList:
        print('\n\n Learner: CEVAE')
        start_time = time.time()
        from cevae import CEVAE as CEVAE
        print('Note: Treatments should be the first columns of X')
        ParamsList['CEVAE']['epochs'] = trykey(ParamsList['CEVAE'], 'epochs', 100)
        ParamsList['CEVAE']['batch'] = trykey(ParamsList['CEVAE'], 'batch', 200)
        ParamsList['CEVAE']['z_dim'] = trykey(ParamsList['CEVAE'], 'z_dim', 5)

        confeatures, binfeatures = [],[]
        for col in range(X_train01.shape[1]):
            a = X_train01.iloc[:,col]
            if not ((a == 0) | (a == 1)).all():
              confeatures.append(col)
            else:
              binfeatures.append(col)

        print('... length con and bin features', len(confeatures), len(binfeatures))
        model_cevae = CEVAE(X_train01, X_test01, y_train, y_test, TreatCols, 
                            binfeats=binfeatures, contfeats=confeatures,
                            epochs=ParamsList['CEVAE']['epochs'], 
                            batch=ParamsList['CEVAE']['batch'],
                            z_dim=ParamsList['CEVAE']['z_dim'])
        cate = model_cevae.fit_all()
        coef_table['CEVAE'] = cate
        times['CEVAE'] = time.time() - start_time
        print('Done!')

    if not timeit:
        return coef_table
    else:
        return coef_table, times

def organize_output(experiments, true_effect, exp_time):
    experiments['TrueTreat'] = true_effect
    experiments.set_index('causes', inplace=True)
    BaselinesNames = experiments.columns

    mae = []
    for col in BaselinesNames:
        mae.append(np.sum(np.abs(experiments[col] - experiments['TrueTreat'])) / experiments.shape[0])

    output = pd.DataFrame({'Method': BaselinesNames, 'MAE': mae})
    exp_time['TrueTreat'] = 0
    output['Time(s)'] = [exp_time[m] for m in output['Method'].values]
    print(experiments, '\n', output)
    return experiments, output

def trykey(params,key,default):
    try:
        return params[key]
    except KeyError:
        params[key] = default
        return params[key]

In [ ]:
params = {'DA':{'k':[15]},
          'BART':{'n_trees':15,'n_burn':100},
          'CEVAE':{'num_epochs':100,'batch': 200,'z_dim':10},
          'seed':10}

sdata_gwas = gwas_simulated_data(prop_tc=0.05, seed=params['seed'], pca_path='/content/CompBioAndSimulated_Datasets/data/tgp_pca2.txt')
X, y, y01, treatement_columns, treatment_effects,group  = sdata_gwas.generate_samples()

#'DA','BART'
experiments, exp_time = baselines(['CEVAE','DA','BART'],pd.DataFrame(X),y01, params,
                       TreatCols = treatement_columns, timeit=True, seed=params['seed']) #'CEVAE',

experiments, output = organize_output(experiments, treatment_effects[treatement_columns], exp_time)

In [ ]:
params = {'DA':{'k':[15], 'class_weight':{0:1, 1:1}},
          'BART':{'n_trees':3,'n_burn':50},
          'CEVAE':{'num_epochs':100,'batch': 200,'z_dim':5},
          'seed':10}
          
sdata_copula = copula_simulated_data(seed = params['seed'])
X, y, y01, treatement_columns, treatment_effects = sdata_copula.generate_samples()

experiments, exp_time = baselines(['BART','DA','CEVAE'],pd.DataFrame(X),y01, params,
                       TreatCols = treatement_columns, timeit=True) #'CEVAE',

experiments, output = organize_output(experiments, treatment_effects[treatement_columns], exp_time)

In [ ]:
# ADD IHDP DATASET 

In [ ]:
a = np.array([0.176609,0.057250 ,0.043792 ])
b = np.array([0.027523,-0.743119,0.042752 ])
np.sum(np.abs(a-b))/2